In [1]:
import os
from dotenv import load_dotenv

from src.utils import read_yaml


load_dotenv()


# os.environ['GOOGLE_API_KEY'] = os.getenv('GEMINI_API_KEY')
os.environ['TAVILY_API_KEY'] = os.getenv("TAVILY_API_KEY")
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")

config = read_yaml('config.yaml')
config

{'llm': {'provider': 'groq',
  'settings': {'temperature': 1.0, 'max_tokens': 128}},
 'tools': {'search': {'provider': 'tavily',
   'settings': {'max_results': 5, 'search_depth': 'basic'}}}}

In [2]:
from src.nodes import Planner, Solver, Workers

task = "what is the hometown of the 2024 australian open winner"

planner = Planner(config=config['llm'])
solver  = Solver(config=config['llm'])

search_engine = Workers.call_engine(name="search",
                                            config=config['tools']['search'])
llm_engine = Workers.call_engine(name="llm", 
                                        config=config['llm'])

regex_pattern = r"Plan:\s*(.+)\s*(#E\d+)\s*=\s*(\w+)\s*\[([^\]]+)\]"

In [3]:
planning = planner.run(task)
print(planning.content)

Plan: Use Tavily to find out who won the 2024 Australian Open. #E1 = Tavily[2024 Australian Open winner]
Plan: Utilize LLM to determine the hometown of the tennis player who won the tournament. #E2 = LLM[What is the hometown of #E1?]


In [4]:
import re

matches = re.findall(regex_pattern, planning.content)
temp_workers =  {"steps": matches, "plan_string": planning.content} # _, step_name, tool, tool_input 
temp_workers

{'steps': [('Use Tavily to find out who won the 2024 Australian Open. ',
   '#E1',
   'Tavily',
   '2024 Australian Open winner'),
  ('Utilize LLM to determine the hometown of the tennis player who won the tournament. ',
   '#E2',
   'LLM',
   'What is the hometown of #E1?')],
 'plan_string': 'Plan: Use Tavily to find out who won the 2024 Australian Open. #E1 = Tavily[2024 Australian Open winner]\nPlan: Utilize LLM to determine the hometown of the tennis player who won the tournament. #E2 = LLM[What is the hometown of #E1?]'}